In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing

In [ ]:
from google.colab import files
data = files.upload()

Saving Final_data.csv to Final_data.csv


In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_data(rev):

    removelist = ""
    rev = re.sub('','',rev)          #remove HTML tags
    rev = re.sub('https://.*','',rev)   #remove URLs
    rev = re.sub(r'[^w'+removelist+']', ' ',rev)    #remove non-alphanumeric characters


    rev = rev.lower()
    rev = rev.split()
    rev = [lemmatizer.lemmatize(word) for word in rev if word not in set(stopwords.words('english'))]
    rev = ' '.join(rev)

    return rev



In [ ]:
li = []
i = 0
for r in data['review']:
    s = lemmatize_data(r)
    li.append(s)
    i = i+1
    if i % 5000 == 0:
        print(i)


data['Reviews'] = li

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [ ]:
data.drop('review', axis = 1, inplace = True)
data.to_csv('Final_data.csv')


In [ ]:
data = pd.read_csv('Final_data.csv')

In [ ]:
data

,Unnamed: 0,sentiment,Reviews
0,0,positive,one reviewer mentioned watching 1 oz episode h...
1,1,positive,wonderful little production br br filming tech...
2,2,positive,thought wonderful way spend time hot summer we...
3,3,negative,basically family little boy jake think zombie ...
4,4,positive,petter mattei love time money visually stunnin...
...,...,...,...
49995,49995,positive,thought movie right good job creative original...
49996,49996,negative,bad plot bad dialogue bad acting idiotic direc...
49997,49997,negative,catholic taught parochial elementary school nu...
49998,49998,negative,going disagree previous comment side maltin on...


In [ ]:
corpus = []
for sent in data['Reviews']:
    corpus.append(sent)

In [ ]:
df_train = pd.DataFrame(corpus)

In [ ]:
special_tokens = ("<START>", "<END>")
df_train = df_train.apply(lambda x:
                     special_tokens[0]+' '+x+' '+special_tokens[1])


In [ ]:
train = []
for arr in df_train.values:
    train.append(str(arr))


In [ ]:
train_tokenizer = kprocessing.text.Tokenizer(lower=False, split=' ', oov_token=None, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
train_tokenizer.fit_on_texts(train)
X_train_vocab = {"<PAD>":0}
X_train_vocab.update(train_tokenizer.word_index)


lst_text2seq= train_tokenizer.texts_to_sequences(train)

X_train = kprocessing.sequence.pad_sequences(lst_text2seq,
                    maxlen=1200, padding="post", truncating="post")

In [ ]:
X_train[0].shape

(1200,)

In [ ]:
import gensim.downloader as api
nlp = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
X_embeddings = np.zeros((len(X_train_vocab)+1, 300))
for word,idx in X_train_vocab.items():
    ## update the row with vector
    try:
        X_embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [ ]:
X_embeddings.shape

(91561, 300)

In [ ]:
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense, Input, Embedding, Attention,Concatenate, Dropout
from tensorflow.keras import Model

In [ ]:
latent_dim = 300
embedding_dim = 200
max_length = 1200

In [ ]:


# Encoder
encoder_inputs = Input(shape=(max_length, ))

    # Embedding layer
enc_emb = Embedding(len(X_train_vocab), embedding_dim,
                        trainable=True)(encoder_inputs)

    # Encoder LSTM 1
encoder_lstm1 = LSTM(latent_dim, return_sequences=True,
                        return_state=True, dropout=0.4,
                        recurrent_dropout=0.4)
(encoder_output1, state_h1, state_c1) = encoder_lstm1(enc_emb)

    # Encoder LSTM 2
encoder_lstm2 = LSTM(latent_dim, return_sequences=True,
                        return_state=True, dropout=0.4,
                        recurrent_dropout=0.4)
(encoder_output2, state_h2, state_c2) = encoder_lstm2(encoder_output1)

    # Encoder LSTM 3
encoder_lstm3 = LSTM(latent_dim, return_state=True,
                        return_sequences=True, dropout=0.4,
                        recurrent_dropout=0.4)
(encoder_outputs, state_h, state_c) = encoder_lstm3(encoder_output2)

    # Set up the decoder, using encoder_states as the initial state
decoder_inputs = Input(shape=(None, ))

    # Embedding layer
dec_emb_layer = Embedding(len(X_train_vocab), embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

    # Decoder LSTM
decoder_lstm = LSTM(latent_dim, return_sequences=True,
                        return_state=True, dropout=0.4,
                        recurrent_dropout=0.2)
(decoder_outputs, decoder_fwd_state, decoder_back_state) = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

    # Dense layer
decoder_dense = TimeDistributed(Dense(len(X_train_vocab), activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
generator_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Inference Models

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                        state_h, state_c])

    # Decoder setup

    # Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_length, latent_dim))

    # Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_inputs)

    # To predict the next word in the sequence, set the initial states to the states from the previous time step
(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
            initial_state=[decoder_state_input_h, decoder_state_input_c])

    # A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

    # Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                        decoder_state_input_h, decoder_state_input_c],
                        [decoder_outputs2] + [state_h2, state_c2])









In [ ]:
reverse_target_word_index = train_tokenizer.index_word
reverse_source_word_index = train_tokenizer.index_word
target_word_index = train_tokenizer.word_index


In [ ]:
# Inference Models
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                        state_h, state_c])

    # Decoder setup

    # Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_length, latent_dim))

    # Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_inputs)

    # To predict the next word in the sequence, set the initial states to the states from the previous time step
(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
            initial_state=[decoder_state_input_h, decoder_state_input_c])

    # A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

    # Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                        decoder_state_input_h, decoder_state_input_c],
                        [decoder_outputs2] + [state_h2, state_c2])



In [ ]:
def decode_sequence(input_seq):
    (e_out, e_h, e_c) = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))




    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        (output_tokens, h, c) = decoder_model.predict([target_seq]
                + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != '<END>':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find the stop word.
        if sampled_token == '<END>' or len(decoded_sentence.split()) \
            >= max_length - 1:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        (e_h, e_c) = (h, c)


    return decoded_sentence

In [ ]:
# To convert sequence to text
def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0:
            newString = newString + reverse_source_word_index[i] + ' '

    return newString

In [ ]:
import random

In [ ]:
generator_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1200)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1200, 200)    18312000    ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 1200, 300),  601200      ['embedding[0][0]']              
                                 (None, 300),                                                     
                                 (None, 300)]                                                     
                                                                                              

In [ ]:
all_seq =[]
for i in range (5):
    random_inp = []
    for i in range(max_length):
        entry = random.randint(0, len(X_train_vocab))
        random_inp.append(entry)
        input_text = seq2text(random_inp)
    print("Input text :", input_text)
    generated_text = decode_sequence((np.array(random_inp)).reshape(1,max_length))
    print("Generated text :", generated_text)
    print()

    all_seq.append(generated_text)

Input text : wiitches 2520 neversoft uddin farther braxus vincente slezak interpreter podgorska snagging lbp corine blockbust soaker 8pm serra wolliaston quato beamont yourelf domination chalky thomsen no southerrners supercilious langian malinged aranoa treacly tyranosaurous raided dearwood fraternization hideous fanshawe gunpowder dogsbody alselmo schoolmistress istanbul distorto gauguin corfu grimas montauk vitnam talkd willis tenuity yammering hoffbrauhaus soundscape anderssons auds ridiculously jessika stinko swish biciclette cavil gimpel contradicted reckons docking disconforting dfz campiness assiniboines ritual renwick transmitted hindersome combustion via hovered diverged somehow rythmic dooblebop reconsider darkens blunted overbudgeted puzzle trustee mcmichael panjab socialising diamontopolous istemiyorum singsing qualitatively gaynor kayla teinowitz gabonu kavorkian willie lyric carroll explained subzero mostfamous cra lilililililii quiz siberiade unscience gremlin honore da

In [ ]:
demo_tokenizer = kprocessing.text.Tokenizer(lower=False, split=' ', oov_token=None, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
demo_tokenizer.fit_on_texts(all_seq)
generated_vocab = {"<PAD>":0}
generated_vocab.update(demo_tokenizer.word_index)


generated_seq= demo_tokenizer.texts_to_sequences(all_seq)
seq = kprocessing.sequence.pad_sequences(generated_seq,
                    maxlen=1200, padding="post", truncating="post")


In [ ]:
len(generated_vocab)

5080